In [ ]:
import pandas as pd
import numpy as np
import glob
from cigar import Cigar

In [ ]:
c = Cigar("151M1D24M")
cig_dict = dict(c.items())
rev_cig_dict = {}
for k,v in cig_dict.items():
    if v in rev_cig_dict.keys():
        rev_cig_dict[v] += k
    else:
        rev_cig_dict[v] = k
rev_cig_dict

In [ ]:
cd /home/tian/Documents/working/inteins_in_mg/deadn6/read_mapping/competitive_trial/complete_reads/

In [ ]:
at_coords = {}
with open("../../dn6_splice_coords.txt",'r') as ofh:
    for line in ofh:
        line = line.strip()
        seq_splices = []
        if "+" in line:
            multi_line = line.split(",")[1].split("/")
            seq_splices.append(int(multi_line[0]))
            site_index = 1
            while site_index < len(multi_line)-1:
                seq_splices.append(seq_splices[site_index-1] + (int(multi_line[site_index].split("+")[1]) 
                                   - int(multi_line[site_index].split("+")[0])))
                site_index += 1
        else:
            seq_splices.append(int(line.split(",")[1].split("/")[0]))
            
        at_coords[line.split(",")[0]] = seq_splices
        
#gets the spliced extein coordinates for insertion

In [ ]:
at_coords

In [ ]:
#generate file for samtools region calls
for target in at_coords.keys():
    m_intein = 1
    for site in at_coords[target]:
        
        print("extein{}:{}-{}".format(target, site-3,site+3), file=open("at_samtools_coords_ext_reads.txt",'a'))
       # m_intein +=1
    #then i used te following line: 
    #for f in `cat samtools_coords_ext_reads.txt`;do samtools view reads2_dn6_extein.sort.bam $f >> reads2_dn6.iis_only.sam;done


In [ ]:
for target in at_coords.keys():
    m_intein = 1
    for site in at_coords[target]:
        
        print("extein{}_{}\t{}".format(target, m_intein,site), file=open("at_samtools_iis_exact.txt",'a'))
        m_intein +=1
    #then i used te following line: 
    #for f in `cat samtools_coords_ext_reads.txt`;do samtools view reads2_dn6_extein.sort.bam $f >> reads2_dn6.iis_only.sam;done


In [ ]:
at_intein_iis_exact = {}
with open("at_samtools_iis_exact.txt",'r') as ofh:
    for line in ofh:
        at_intein_iis_exact[line.split("\t")[0]] = int(line.split("\t")[1].strip())
at_intein_iis_exact

In [ ]:
at_iis_summary = {}
for key in at_intein_iis_exact.keys():
    at_iis_summary[key] = 0
at_iis_summary

In [ ]:
len(at_iis_summary)

In [ ]:
dn6_iis_sam = pd.read_table('reads3_dn6.iis_only.sam',
                        usecols=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13], header=None, sep='\t')
dn6_iis_sam.columns = ['read', 'bits', 'target_name',
                   'pos', 'mapq', 'cigar', 'mate_target', 'mpos', 'tlen' , "seq", 'nm', 'as', 'xs']
dn6_iis_sam

In [ ]:
#use samtools to extract reads for a position
for intein, junction in at_intein_iis_exact.items():    
    checker = 0
    for indy, row in dn6_iis_sam.iterrows():
        if row['target_name'] != intein[:-2]:
            continue
        if (row["pos"] + 150) - junction < 15: #this cover reads that are not long enough to span insertion site, at least by 15 overhang
            continue
        if junction - row["pos"] < 15: #if the read starts beyond the insertion site, skip
            continue
        if "I" in row["cigar"]:
            I_total =0
            c = Cigar(row['cigar'])
            clist = list(c.items())
            for tupes in clist:
                if tupes[1] == "I":
                    I_total += tupes[0]
            
            
         
            if I_total < 6:
                checker +=1
    at_iis_summary[intein] += checker

        


In [ ]:
at_iis_summary

In [ ]:
for name1, counts in at_iis_summary.items():
    print("{}\t{}".format(name1,counts), file=open("dn6_all_compet.summary.tab","a"))